In [1]:
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255315 sha256=cfc78db0bcd52db93e04dd6fc2c24730caf87ed9059f5188e510dee53387b781
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [2]:
# Import needed libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import pickle5

In [3]:
class MyNeuralNetwork:
    def __init__(self, input_dimension):
        # Define neural network and hidden layers
        self.model = Sequential()
        self.model.add(Dense(1000, input_dim=input_dimension,
                             activation='relu'))
        self.model.add(Dense(1000, activation='relu'))
        self.model.add(Dense(1, activation='linear'))
        self.model.compile(loss='mean_squared_error', optimizer='adam')

In [6]:
# Import data/features from outside files
with open('nn_data/qb_features.pkl', 'rb') as inp:
    qb_x = pickle5.load(inp)
with open('nn_data/qb_values.pkl', 'rb') as inp:
    qb_y = pickle5.load(inp)
with open('nn_data/qb_test_features.pkl', 'rb') as inp:
    qb_x_test_features = pickle5.load(inp)

with open('nn_data/rb_features.pkl', 'rb') as inp:
    rb_x = pickle5.load(inp)
with open('nn_data/rb_values.pkl', 'rb') as inp:
    rb_y = pickle5.load(inp)
with open('nn_data/rb_test_features.pkl', 'rb') as inp:
    rb_x_test_features = pickle5.load(inp)

with open('nn_data/wr_features.pkl', 'rb') as inp:
    wr_x = pickle5.load(inp)
with open('nn_data/wr_values.pkl', 'rb') as inp:
    wr_y = pickle5.load(inp)
with open('nn_data/wr_test_features.pkl', 'rb') as inp:
    wr_x_test_features = pickle5.load(inp)

with open('nn_data/te_features.pkl', 'rb') as inp:
    te_x = pickle5.load(inp)
with open('nn_data/te_values.pkl', 'rb') as inp:
    te_y = pickle5.load(inp)
with open('nn_data/te_test_features.pkl', 'rb') as inp:
    te_x_test_features = pickle5.load(inp)

qb_N = len(qb_y)
rb_N = len(rb_y)
wr_N = len(wr_y)
te_N = len(te_y)
qb_num_features = len(qb_x[0])
rb_num_features = len(rb_x[0])
wr_num_features = len(wr_x[0])
te_num_features = len(te_x[0])

print("Players in seasons 2019-2022:")
print("QB: " + str(qb_N))
print("RB: " + str(rb_N))
print("WR: " + str(wr_N))
print("TE: " + str(te_N))

print("Players in 2023 season:")
print("QB: " + str(len(qb_x_test_features)))
print("RB: " + str(len(rb_x_test_features)))
print("WR: " + str(len(wr_x_test_features)))
print("TE: " + str(len(te_x_test_features)))

print("Number of features for each position:")
print("QB: " + str(qb_num_features))
print("RB: " + str(qb_num_features))
print("WR: " + str(qb_num_features))
print("TE: " + str(qb_num_features))

Players in seasons 2019-2022:
QB: 215
RB: 336
WR: 594
TE: 325
Players in 2023 season:
QB: 83
RB: 146
WR: 225
TE: 119
Number of features for each position:
QB: 675
RB: 675
WR: 675
TE: 675


In [ ]:
# TRAINING/FINDING BEST HYPERPARAMETERS

# Divide the data into 80% training and 20% validation
training_indexes = np.random.choice(np.arange(len(qb_x)), int(qb_N * 0.8), replace=False)
validation_indexes = np.ones((qb_N,), bool)
validation_indexes[training_indexes] = False
x_training = qb_x[training_indexes]
y_training = qb_y[training_indexes]
x_validation = qb_x[validation_indexes]
y_validation = qb_y[validation_indexes]

# Find best batch size
best_batch_size_error = float('inf')
for num_batch_size in range(5, 55, 10):
    # Create the neural network
    neural_network = MyNeuralNetwork(input_dimension=qb_num_features)

    # Fit the neural network
    neural_network.model.fit(x_training, y_training, epochs=5000, batch_size=num_batch_size)

    # Get predictions for validation data
    y_predicted = neural_network.model.predict(x_validation)

    # Calculate the mean squared error
    mse = mean_squared_error(y_validation, y_predicted)
    print('Mean squared error: ' + str(mse))
    if mse < best_batch_size_error:
        best_batch_size_error = mse
        best_batch_size = num_batch_size

print(best_batch_size)

In [ ]:
# TESTING ON 2023 SEASON

# Run neural network and get predictions for each position
positions = ["qb", "rb", "wr", "te"]
for position in positions:
    print(position)
    if position == "qb":
        x = qb_x
        y = qb_y
        x_test_features = qb_x_test_features
    elif position == "rb":
        x = rb_x
        y = rb_y
        x_test_features = rb_x_test_features
    elif position == "wr":
        x = wr_x
        y = wr_y
        x_test_features = wr_x_test_features
    elif position == "te":
        x = te_x
        y = te_y
        x_test_features = te_x_test_features

    # Create and train neural network
    neural_network = MyNeuralNetwork(input_dimension=675)
    neural_network.model.fit(x, y, epochs=5000, batch_size=best_batch_size)

    # Predict for 2023 NFL season
    y_predictions = neural_network.model.predict(x_test_features)
    print(y_predictions)

    # Store predictions in output files
    pkl_file = 'nn_data/' + position + '_predictions.pkl'
    txt_file = 'nn_data/' + position + '_predictions.txt'
    with open(pkl_file, 'wb') as outp:
        pickle5.dump(y_predictions, outp, pickle5.HIGHEST_PROTOCOL)
    with open(txt_file, "w") as file:
        file.write(str(y_predictions))

Streaming output truncated to the last 5000 lines.
Epoch 2561/5000
33/33 [==============================] - 1s 26ms/step - loss: 10.2785
Epoch 2562/5000
33/33 [==============================] - 1s 27ms/step - loss: 11.4884
Epoch 2563/5000
33/33 [==============================] - 1s 27ms/step - loss: 11.7602
Epoch 2564/5000
33/33 [==============================] - 1s 25ms/step - loss: 8.1482
Epoch 2565/5000
33/33 [==============================] - 1s 28ms/step - loss: 5.7088
Epoch 2566/5000
33/33 [==============================] - 1s 27ms/step - loss: 9.9715
Epoch 2567/5000
33/33 [==============================] - 1s 26ms/step - loss: 18.0954
Epoch 2568/5000
33/33 [==============================] - 1s 30ms/step - loss: 25.9295
Epoch 2569/5000
33/33 [==============================] - 1s 33ms/step - loss: 14.1572
Epoch 2570/5000
33/33 [==============================] - 1s 33ms/step - loss: 10.2698
Epoch 2571/5000
33/33 [==============================] - 1s 32ms/step - loss: 9.9497
Epoch 2